In [1]:
import json
import yaml
import boto3
import logging
from globals import *
from pathlib import Path
from botocore.exceptions import ClientError

In [2]:
# Setup logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
!pygmentize globals.py

# global constants
CONFIG_FILE: str = "config.yml"
MODEL_ID_TOPROMPT_ID_MAPPING_FILE: str = ".model_id_to_prompt_id_mapping.json"

LAMBDA_DIR: str = "lambda"
CONFIG_FILE: str = "config.yml"
CODE_GEN_LAMBDA: str = "code-gen"
LAMBDA_ARN_FILE: str = ".lambda_arn"


In [4]:
config = yaml.safe_load(Path(CONFIG_FILE).read_text())
logger.info(f"config=\n{json.dumps(config, indent=2)}")

[2025-01-19 00:49:59,651] p21015 {3811199622.py:2} INFO - config=
{
  "general": {
    "app_name": "code-gen-agent",
    "role_name": "CodeGenLambdaRole",
    "region": "us-east-1"
  },
  "prompt_info": {
    "name": "USACO_{model_id}",
    "description": "Generate code for the USACO benchmark"
  },
  "prompt_templates": {
    "nova": {
      "models": [
        "amazon.nova-pro-v1:0",
        "amazon.nova-lite-v1:0",
        "amazon.nova-micro-v1:0"
      ],
      "text": "Please reply with a Python 3 solution to the below problem. Read the general instructions below that are applicable to every task,\n\n{{question}}\n\nImportant instructions to follow:\n\n1. Ensure that your code is wrapped in '```python' and '```' Markdown delimiters.\n2. Provide exactly one block of code containing the entire solution.\n3. The code should include a main function and an `if __name__ == \"__main__\"` block as the entry point.\n4. Begin by reasoning through the problem and conceptualizing a solution. 

In [5]:
# fetch the current AWS region
region = config['general']['region']
# the region to be dynamically fetched
logger.info(f"Current AWS region: {region}")
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)

[2025-01-19 00:49:59,659] p21015 {4102934896.py:4} INFO - Current AWS region: us-east-1
[2025-01-19 00:49:59,683] p21015 {credentials.py:1075} INFO - Found credentials from IAM Role: fmbench-orchestrator


In [6]:
import re
from typing import List

def extract_mustache_keys(text: str) -> List[str]:
    """
    Extract all mustache-style keys ({{key}}) from the input text.
    
    Args:
        text (str): Input text containing mustache syntax
        
    Returns:
        List[str]: List of extracted keys without the curly braces
    """
    # The pattern looks for:
    # \{\{ - literal {{ (escaped because { is special in regex)
    # (.*?) - any characters (non-greedy match)
    # \}\} - literal }}
    pattern = r'\{\{(.*?)\}\}'
    
    # Find all matches and extract the group inside the braces
    matches = re.findall(pattern, text)
    
    # Strip whitespace from each match
    return [match.strip() for match in matches]

In [7]:
def create_or_update_bedrock_prompt(name: str, description: str, text: str, parameters: dict, model_id: str):
    """
    Creates a prompt configuration for Amazon Bedrock using specified parameters.
    
    This function constructs a prompt template with inference settings and template configuration
    for use with Amazon Bedrock's language models. It automatically extracts input variables 
    from mustache-style placeholders in the template text.

    Args:
        name (str): The name identifier for the prompt template
        description (str): A description of the prompt template's purpose
        text (str): The template text containing mustache-style variables (e.g., {{variable}})
        parameters (dict): Configuration parameters including:
            - max_tokens (int): Maximum number of tokens in the response
            - temperature (float): Sampling temperature for text generation
        model_id (str): The identifier of the Bedrock model to use

    Returns:
        dict: The response from the Bedrock create_prompt API call, containing the created
              prompt configuration details

    Examples:
        >>> parameters = {
        ...     'max_tokens': 100,
        ...     'temperature': 0.7
        ... }
        >>> create_or_update_bedrock_prompt(
        ...     name="test_prompt",
        ...     description="A test prompt",
        ...     text="Hello {{name}}, how are you?",
        ...     parameters=parameters,
        ...     model_id="anthropic.claude-v2"
        ... )

    Notes:
        - The function automatically creates a default variant named "default_variant"
        - Input variables are automatically extracted from {{mustache}} syntax in the template text
        - Logs the API response using the logger module
        - Requires the bedrock_agent and logger to be properly configured
    """
    
    default_variant_name = "default_variant"
    input_variables = [dict(name=k) for k in extract_mustache_keys(text)]
    variant = {
            "inferenceConfiguration": {
            "text": {
                "maxTokens": parameters['max_tokens'],
                "temperature": parameters['temperature'],
                }
            },
            "modelId": model_id,
            "name": default_variant_name,
            "templateConfiguration": {
                "text": {
                    "inputVariables": input_variables,
                    "text": text
                }
            },
            "templateType": "TEXT"
        }
    
    try:
        response = bedrock_agent.create_prompt(name=name,
                                               description=description,
                                               variants=[variant],
                                               defaultVariant=default_variant_name)
    except ClientError as e:
        logger.error(f"exception occured while creating prompt, exception={e}")
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        
        # Check for ConflictException
        if error_code == 'ConflictException':
            logger.error(f"got {error_code} exception, error_message={error_message}, going to update the prompt")
            # in case of prompt already exists the error messages looks like this
            # Couldn't perform CreatePrompt operation. The name USACO_amazon-nova-pro-v1-0 already exists for id FPPQT96U8Y. Retry your request with a different name., going to update the prompt"
            # so we can extract the id from the error message using the following regex
            pattern = r"exists for id ([A-Z0-9]+)\."
            match = re.search(pattern, error_message)
  
            if match:
                prompt_id = match.group(1)
                logger.info(f"id for the prompt that already exists is {prompt_id}")
                response = bedrock_agent.update_prompt(promptIdentifier=prompt_id,
                                                       name=name,
                                                       description=description,
                                                       variants=[variant],
                                                       defaultVariant=default_variant_name)
                logger.info(f"response after updating prompt = {response}")
            else:
                raise
    logger.info(f"response={json.dumps(response, indent=2, default=str)}")
    return response


    


In [8]:
model_id_to_prompt_mapping = {}
for model_family, info in config['prompt_templates'].items():
    for model_id in info['models']:
        name = config['prompt_info']['name'].format(model_id=model_id)
        name = re.sub('[:\.]', '-', name)
        try:
            response = create_or_update_bedrock_prompt(name,
                                                       config['prompt_info']['description'],
                                                       info['text'],
                                                       config['inference_parameters'],
                                                       model_id)
            model_id_to_prompt_mapping[model_id] = response['id']
        except Exception as e:
            logger.error(f"exception occurred while creating prompt, name={name}, exception={e}")

<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_21015/2351719983.py:5: SyntaxWarning: invalid escape sequence '\.'
  name = re.sub('[:\.]', '-', name)
[2025-01-19 00:49:59,900] p21015 {1954817771.py:68} ERROR - exception occured while creating prompt, exception=An error occurred (ConflictException) when calling the CreatePrompt operation: Couldn't perform CreatePrompt operation. The name USACO_amazon-nova-pro-v1-0 already exists for id FPPQT96U8Y. Retry your request with a different name.
[2025-01-19 00:49:59,901] p21015 {1954817771.py:74} ERROR - got ConflictException exception, error_message=Couldn't perform CreatePrompt operation. The name USACO_amazon-nova-pro-v1-0 already exists for id FPPQT96U8Y. Retry your request with a different name., going to update the prompt
[2025-01-19 00:49:59,901] p21015 {1954817771.py:83} INFO - id for the prompt that already exists is FPPQT96U8Y
[2025-01-19 00:50:00,051] p21015 {195481

In [9]:
logger.info(json.dumps(model_id_to_prompt_mapping, indent=2))

[2025-01-19 00:50:01,153] p21015 {2717539926.py:1} INFO - {
  "amazon.nova-pro-v1:0": "FPPQT96U8Y",
  "amazon.nova-lite-v1:0": "RJG7KU4HDN",
  "amazon.nova-micro-v1:0": "P2CS53D4NC",
  "us.anthropic.claude-3-5-haiku-20241022-v1:0": "6FM16J82V8",
  "us.anthropic.claude-3-5-sonnet-20241022-v2:0": "4XQSDFVR7I"
}


In [10]:
Path(MODEL_ID_TOPROMPT_ID_MAPPING_FILE).write_text(json.dumps(model_id_to_prompt_mapping, indent=2))

252

In [11]:
bedrock_agent.get_prompt(promptIdentifier='6FM16J82V8')['variants'][0]#['templateConfiguration']['text']

{'inferenceConfiguration': {'text': {'maxTokens': 2000,
   'temperature': 0.699999988079071}},
 'modelId': 'us.anthropic.claude-3-5-haiku-20241022-v1:0',
 'name': 'default_variant',
 'templateConfiguration': {'text': {'inputVariables': [{'name': 'question'}],
   'text': 'Please reply with a Python 3 solution to the below problem. Read the general instructions below\nthat are applicable to every task,\n\n<instructions>\n1. Make sure to wrap your code in \'```python\' and \'```\' Markdown delimiters.\n2. Include exactly one block of code with the entire solution.\n3. The code should always have main function and an if __name__ == "__main__" block as the entrypoint.\n4. Reason through the problem and conceptualize a solution first, then write pseudocode, and finally output the Python with your solution steps in comments.\n5. Carefully examine the SAMPLE INPUT: and SAMPLE OUTPUT: sections provided as part of the problem and follow the format exactly as specified there.\n6. There might be e